<a href="https://colab.research.google.com/github/HarshaVardhanLanka/springboard_project/blob/main/Day_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U google-generativeai tavily-python pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.5/329.5 kB 11.3 MB/s eta 0:00:00


In [3]:
import os
import google.generativeai as genai
from tavily import TavilyClient
from google.colab import userdata
from google.colab import files
import pypdf
import io

# 1. Setup API Keys
try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    TAVILY_API_KEY = userdata.get('TAVILY')
except:
    print("Error: Please set your API keys in the Colab Secrets tab.")

# 2. Configure Gemini
genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel('gemini-1.5-flash')

# 3. Configure Tavily
tavily = TavilyClient(api_key=TAVILY_API_KEY)

print("Setup complete.")

Setup complete.


In [6]:
def get_pdf_text(uploaded_file):
    """Extracts text from a PDF binary."""
    pdf_reader = pypdf.PdfReader(io.BytesIO(uploaded_file))
    text = ""
    for page in pdf_reader.pages:
        text += page.extract_text() + "\n"
    return text

def generate_plan(source_text, mode="topic"):
    """
    Generates search queries based on the mode.
    Mode 'topic': Breaks down a general topic.
    Mode 'pdf': Finds gaps/validation points in a paper.
    """
    print(f"🤔 Generating research plan for mode: {mode}...")

    if mode == "topic":
        prompt = f"""
        You are a research planner. Break down this topic into 3 distinct, searchable questions.
        Topic: {source_text}
        Output: Return ONLY 3 questions separated by newlines. No numbering.
        """
    else: # mode == pdf
        prompt = f"""
        You are a Senior Researcher. Read the following paper excerpt and identify 3 questions
        to validate the claims or find the latest state-of-the-art comparisons.

        Paper Excerpt: {source_text[:15000]}

        Output: Return ONLY 3 search queries separated by newlines. No numbering.
        """

    response = model.generate_content(prompt)
    return [q.strip() for q in response.text.strip().split('\n') if q.strip()]

def perform_research(questions):
    """Executes the search for the generated questions."""
    print(f"🔎 Searching the web...")
    results = []
    for q in questions:
        print(f"   - Searching: {q}")
        try:
            # Using advanced search depth for better quality
            res = tavily.search(query=q, search_depth="advanced", max_results=2)
            content = "\n".join([r['content'] for r in res['results']])
            results.append(f"Query: {q}\nFindings: {content}")
        except Exception as e:
            print(f"Error searching {q}: {e}")
    return "\n\n".join(results)

def synthesize_report(source_text, research_data, mode="topic"):
    """Summarizes everything into a final report."""
    print(f"📝 Synthesizing final report...")

    if mode == "topic":
        prompt = f"""
        Write a comprehensive research summary on the topic below, incorporating the search findings.

        User Topic: {source_text}
        Search Data: {research_data}

        Format: Markdown with headers.
        """
    else:
        prompt = f"""
        Create a "Deep Dive" analysis of the provided research paper using the external search data.

        1. Summarize the Paper's core contribution.
        2. Compare it with the "External Validation" findings (State-of-the-art, updates, or contradictions).
        3. Provide a final conclusion on the paper's relevance today.

        Paper Text (Context): {source_text[:15000]}
        External Validation Data: {research_data}

        Format: Markdown with headers.
        """

    response = model.generate_content(prompt)
    return response.text

In [12]:
from IPython.display import Markdown, display

print("🤖 RESEARCH AGENT STARTED")
print("Select Mode:")
print("1. Research a Topic (Enter text)")
print("2. Analyze a Paper (Upload PDF)")

choice = input("Enter 1 or 2: ")

source_text = ""
mode = ""

if choice == '1':
    mode = "topic"
    source_text = input("\nEnter your research topic: ")
    if not source_text:
        print("❌ Topic cannot be empty.")
        mode = None

elif choice == '2':
    mode = "pdf"
    print("\n⬆️ Please upload your PDF file now:")
    uploaded = files.upload()
    if uploaded:
        filename = next(iter(uploaded))
        print(f"Processing {filename}...")
        source_text = get_pdf_text(uploaded[filename])
    else:
        print("❌ No file uploaded.")
        mode = None
else:
    print("❌ Invalid choice.")

# Run the workflow if inputs are valid
if mode and source_text:
    print("-" * 50)

    # Fix: Re-initialize the model to a commonly available one
    # The 'gemini-1.5-flash' model might not be available or supported for generateContent in your region/configuration.
    # Changing it to 'gemini-flash-latest' for broader compatibility based on the list of available models.
    model = genai.GenerativeModel('gemini-flash-latest')

    # Step 1: Plan
    queries = generate_plan(source_text, mode=mode)
    print(f"📋 Plan: {queries}")

    # Step 2: Act
    web_data = perform_research(queries)

    # Step 3: Synthesize
    final_output = synthesize_report(source_text, web_data, mode=mode)

    print("\n" + "="*20 + " FINAL REPORT " + "="*20 + "\n")
    display(Markdown(final_output))


🤖 RESEARCH AGENT STARTED
Select Mode:
1. Research a Topic (Enter text)
2. Analyze a Paper (Upload PDF)
Enter 1 or 2: 2

⬆️ Please upload your PDF file now:


Saving 2511.17546v1.pdf to 2511.17546v1.pdf
Processing 2511.17546v1.pdf...
--------------------------------------------------
🤔 Generating research plan for mode: pdf...


ERROR:tornado.access:503 POST /v1beta/models/gemini-flash-latest:generateContent?%24alt=json%3Benum-encoding%3Dint (::1) 5869.66ms


📋 Plan: ['Latest energy resolution and efficiency of cryogenic microcalorimeters for muonic x-ray spectroscopy (10-40 keV)', 'State-of-the-art EFG calculation accuracy open-shell transition elements $Z=23-30$ nuclear quadrupole moments', 'High resolution muonic atom x-ray spectroscopy techniques 10-40 keV comparison']
🔎 Searching the web...
   - Searching: Latest energy resolution and efficiency of cryogenic microcalorimeters for muonic x-ray spectroscopy (10-40 keV)
   - Searching: State-of-the-art EFG calculation accuracy open-shell transition elements $Z=23-30$ nuclear quadrupole moments
   - Searching: High resolution muonic atom x-ray spectroscopy techniques 10-40 keV comparison
📝 Synthesizing final report...

==================== FINAL REPORT ====================



## Deep Dive Analysis: Reference Quadrupole Moments from Lamb Shifts in Muonic Atoms

### 1. Summary of the Paper's Core Contribution

The paper "Reference Quadrupole Moments of Transition Elements from Lamb Shifts in Muonic Atoms" proposes a novel experimental and theoretical approach to significantly reduce the uncertainty associated with the absolute spectroscopic electric quadrupole moments ($Q_{ref}$) of light transition elements ($Z=23$ to $Z=30$, e.g., V, Cr, Mn, Co, Cu).

**The central problem addressed is:** Currently, $Q_{ref}$ values for these elements are derived from electronic hyperfine structure measurements. This method requires calculating the Electric Field Gradient (EFG) created by the electrons at the nucleus. For open-shell transition elements, these EFG calculations are highly challenging due to complex electron correlation and shielding effects, resulting in large uncertainties (Figure 1).

**The proposed solution involves three key components:**

1.  **Muonic Atoms:** By replacing an electron with a heavy muon ($\mu^-$), the complexity of the EFG calculation is drastically simplified (muonic EFG uncertainty is $\approx 1\%$), and the resulting energy splitting is enhanced by $\approx 10^7$.
2.  **Target Transition:** Measurement focuses on the weakly populated $2s_{1/2} \rightarrow 2p_{3/2}$ Lamb shift transition, which falls in the 10–40 keV X-ray energy range.
3.  **Cryogenic Microcalorimeters (MCs):** These detectors provide the requisite high efficiency and excellent energy resolution ($\approx 10$ eV) needed to resolve the small but measurable quadrupole hyperfine splitting in the muonic X-rays (Table I shows current uncertainties in the splitting $B$ are 18–100 eV).

Through detailed calculations and simulations (e.g., optimizing muon momentum and estimating cascade rates for ${}^{63}\text{Cu}$), the authors demonstrate the feasibility of reducing the uncertainty in $Q_{ref}$ by up to an order of magnitude within a single day of measurement. These precise reference moments are essential inputs for nuclear structure studies and quantum chemistry benchmarking.

***

### 2. Comparison with External Validation Findings

The external search data validates the core technological requirements and theoretical motivation behind the proposed methodology, confirming that the paper is situated within the current frontier of nuclear and atomic physics.

| Paper Requirement | External Validation Data | Comparison/Validation Status |
| :--- | :--- | :--- |
| **High-Resolution Detection (10-40 keV)** | Cryogenic Microcalorimeters (MCs) are confirmed as state-of-the-art, increasing relative energy resolution by "up to a factor of 20" over conventional techniques. | **Strong Validation:** The resolution required by the paper (10–20 eV) is entirely consistent with the exceptional performance demonstrated by modern MCs. This technology is capable of resolving the proposed hyperfine structure splitting. |
| **High Muon Flux/Rates** | External findings mention the need for "high-intensity muon beam at CiADS" for advanced measurements. | **Validation of Challenge:** The paper explicitly acknowledges the low population of the $2s$ state and performs optimization based on available rates (e.g., 15 kHz at PSI). External data reinforces that intense muon sources are crucial for pushing the boundary of muonic X-ray spectroscopy. |
| **Need for Simplified EFG** | External data confirms that EFG calculations in electronic systems remain complex, often requiring detailed tensor analysis and specific consideration of materials (cubic lattices, etc.). | **Validation of Motivation:** The external findings reinforce the paper's primary theoretical motivation: EFG calculation in complex systems is challenging and limits accuracy. The shift to muonic atoms successfully bypasses this difficulty. |
| **Relevance of Muonic Spectroscopy** | Muonic X-ray Emission Spectroscopy (MXES) is recognized for its utility in precise QED tests, fundamental constant extraction, and sensitivity to electronic structure. | **Contextual Validation:** The proposed experiment is a highly specialized, precision application within the established, important field of MXES, focusing specifically on the low-energy Lamb shift to extract nuclear properties, rather than the higher-energy transitions (e.g., 1.2 MeV Kα) typically cited for elemental analysis. |

**Synthesis:**

The external data confirms that the paper's proposal relies on sound, advancing technologies (MCs and high-intensity muon beams) and addresses a well-recognized theoretical bottleneck (EGF uncertainty in electronic systems). There are no direct contradictions; rather, the external information validates the technological feasibility and the high relevance of the underlying physics problem.

***

### 3. Final Conclusion on the Paper's Relevance Today

The research paper by Rathi et al. is **highly relevant** in the current landscape of nuclear and atomic physics.

**Key reasons for enduring relevance:**

1.  **Addressing a Fundamental Accuracy Gap:** The high uncertainty in $Q_{ref}$ for light transition elements ($Z=23-30$) remains a limiting factor in nuclear structure studies across entire isotopic chains. This paper offers a clear, actionable experimental pathway to eliminate the theoretical dependency on complex electronic EFG calculations.
2.  **Technological Maturity:** The necessary technological components—specifically Cryogenic Microcalorimeters—are mature and actively utilized in cutting-edge spectroscopy, as confirmed by the external data. The paper correctly leverages their superior resolution (10 eV) to resolve hyperfine splitting previously impossible with solid-state detectors.
3.  **High Impact for Nuclear Theory:** A precise determination of $Q_{ref}$ allows for more stringent benchmarking of theoretical models describing collective motion and deformation in the nucleus. This is particularly crucial for open-shell elements where current models struggle.

In conclusion, this paper outlines a critical next step in high-precision muonic spectroscopy. By combining the unique atomic physics of muonic Lamb shifts with state-of-the-art high-resolution detectors, the method promises to deliver foundational nuclear data that remains inaccessible through conventional means, cementing its contemporary importance.